# Text preprocessing - replace umlauts

In [1]:
news = ['Afrikanische Schweinepest in Deutschland nachgewiesen', 'Amerika entzieht mehr als 1000 chinesischen Studenten und Forschern ihre Visa', 'Brand im Lager Moria: Statisten in einem Abschreckungsdrama', 'CDU-Parteivorsitz: Wie Nord Stream 2 die Karten neu mischt', 'Conti, Schaeffler & Co.: Autozulieferer in der Corona-Falle', 'Debatte nach Moria-Brand: CDU-Abgeordneter fordert, 5000 Migranten aufzunehmen', 'Debatte um Sanktionen: Warum Nord Stream 2 fuer Russland so wichtig ist', 'Deep Learning alleine reicht nicht', 'EZB entscheidet ueber weiteres Vorgehen in Corona-Krise', 'Fiskus hat erst 1,1 Milliarden Euro zurueckgeholt', 'Kommentar: Ein offizielles Corona-Gedenken ist riskant', 'Trump ueber Corona-Risiko: „Ich wollte es immer herunterspielen“', 'Venezuelas Opposition streitet: „Regierung spielen“ oder an Wahlen teilnehmen?', 'Verdacht auf Afrikanische Schweinepest in Brandenburg', 'Vielfaeltige Videoplattform: Wie Youtube Arbeitsplaetze und Wirtschaftswachstum schafft', 'Wahl in NRW: „Corona hat gezeigt, wie wichtig die Kommunen sind“', 'Weniger Firmenpleiten im Halbjahr', 'Woodwards Trump-Buch: Luegen und Protzen', 'Afrikanische Schweinepest in Deutschland: erster Fall in Deutschland bestaetigt', 'Alec Baldwin: Fuenftes Baby mit Ehefrau Hilaria ist da', 'Alexej Nawalny: Mike Pompeo vermutet "hohe russische Regierungsmitglieder" hinter Anschlag', 'Alexej Nawalny: Polizei verschaerft Bewachung des Kremlkritikers in Berliner Charité', 'Barfuss zum Olympiasieg: Wie Abebe Bikila 1960 Marathon-Gold gewann', 'Belarus: Marija Kolesnikowa soll verhoert werden', 'Boom trotz Coronakrise: Bauhauptgewerbe stellt kraeftig ein', 'Corona und Globalisierung: Was Deutschland zu verlieren hat', 'Corona-News am Donnerstag: Hochrechnung: Moeglicherweise neunmal mehr Faelle in USA als bisher bekannt', 'Donald Trump in Bedraengnis nach Woodward-Enthuellungen: "Ich bin ein Cheerleader fuer dieses Land"', 'Donald Trump und Coronavirus: Watergate-Enthueller Bob Woodward bringt Trump in Not', 'Drew Barrymore will nicht mehr heiraten', 'Fernradwege: Die Sucht nach Hoehenmetern - Touren zum Auspowern', 'Formel 1 : Sebastian Vettel faehrt ab 2021 fuer Aston Martin', 'Jan Boehmermann-Twitter-Buch: Gefolgt von niemandem, dem du folgst - Kritik', 'Kassel: Tankstellen-Mitarbeiterin vertreibt Raeuber mit Kuchenblech', 'Moria nach der Brandkatastrophe: Obdachlos in Europa', 'Moria: Gerd Mueller fordert Aufnahme von 2000 Migranten aus Moria', 'Moria: Unbegleitete Minderjaehrige werden aufs griechische Festland gebracht', 'Moria: Unionsabgeordnete fordern nach Brand Aufnahme von 5000 Fluechtlingen', 'Royal Enfield Himalayan: Treue Wanderfreundin', 'Saenger Ronald Bell von Kool & the Gang ist tot', 'San Francisco: Waldbraende in Kalifornien sorgen fuer apokalyptische Stimmung', 'Schweinepest: Agrarministerin Julia Kloeckner zu moeglichem Fall in Deutschland - Livestream', 'Stillgelegte Gleise: Wo die Deutsche Bahn wieder fahren soll', 'US Open: Dominic Thiem debuetiert in der Runde der besten Vier', 'US-Demokraten warnen Boris Johnson vor Bruch des Brexit-Abkommens', 'USA streichen mehr als tausend Visa fuer Forscher aus China', 'USA: Weisses Haus spielte laut Whistleblower russische Wahleinmischung herunter', 'Venezuela: Freddy Guevara verlaesst nach Begnadigung chilenische Botschaft']

In [2]:
def umlauts(word):
    """
    Replace umlauts for a given text
    
    :param word: text as string
    :return: manipulated text as str
    """
    
    tempVar = word  
    
    tempVar = tempVar.replace('ä', 'ae')
    tempVar = tempVar.replace('ö', 'oe')
    tempVar = tempVar.replace('ü', 'ue')
    tempVar = tempVar.replace('Ä', 'Ae')
    tempVar = tempVar.replace('Ö', 'Oe')
    tempVar = tempVar.replace('Ü', 'Ue')
    tempVar = tempVar.replace('ß', 'ss')
    
    return tempVar

In [ ]:
with open('test.txt', encoding='utf-8') as file:
    text = file.read()
text = umlauts(text)
text = ' '.join((text.strip('\n').split()))

text_file = open("test.txt", "w")
n = text_file.write(text)
text_file.close()

# hier beginnt die eigentliche BD-Anwendung

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [ ]:
# if we would use kubernetes then run this instead of next cell
'''conf=SparkConf()\
        .setMaster(local[*])\
        .setappName("WordCount")\
        .setExecutorEnv("spark.executor.memory","4g")\
        .setExecutorEnv("spark.driver.memory","4g")
  spark=SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()'''

In [4]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("WordCount")\
    .getOrCreate()
sc=spark.sparkContext

In [6]:
#hier eine txt der Datenbank einhängen für den wordcount
#file = 'test.txt'
df = sc.parallelize(news)

In [8]:
# exclude punctuation
def lower_clean_str(x):
    punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
    lowercased_str = x.lower()
    for ch in punc:
        lowercased_str = lowercased_str.replace(ch, '')
    return lowercased_str

In [9]:
#remove punktuation and transform to lowercase
df = df.map(lower_clean_str) 

In [11]:
#split sentences into list of words
df = df.flatMap(lambda satir: satir.split(" ")) 

In [12]:
# exclude whitespaces
df = df.filter(lambda x:x!='') 

In [62]:
# count how many times each word occurs
count = df.map(lambda word:(word,1))
countRBK = count.reduceByKey(lambda x,y:(x+y)).sortByKey()

In [63]:
# rank words
countRBK = countRBK.map(lambda x:(x[1],x[0]))
countRBK = countRBK.sortByKey(False)

In [64]:
from nltk.corpus import stopwords
stopwords =stopwords.words('german')

german_stopwords = []
for word in stopwords:
    german_stopwords.append(umlauts(word))

countRBK = countRBK.filter(lambda x: x[1] not in german_stopwords)

In [111]:
import pandas as pd
dff = pd.DataFrame(columns=['trend-word'])
for i in range(5):
    dff = dff.append({'trend-word': countRBK.take(5)[i][1]}, ignore_index=True)

In [112]:
dff

,trend-word
0,moria
1,deutschland
2,mehr
3,schweinepest
4,trump
